In [1]:
#!pip install lightgbm

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset = pd.read_csv("clean_mall_data.csv")
dataset = dataset.drop("Shop_ID", axis=1)
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)

indep_X = dataset.drop("Rent", axis=1)
dep_Y = dataset["Rent"]

In [4]:
def selectkbest(indep_X, dep_Y, n):
    selector = SelectKBest(score_func=f_regression, k=n)
    X_new = selector.fit_transform(indep_X, dep_Y)
    return X_new

def split_scalar(indep_X, dep_Y):
    X_train, X_test, Y_train, Y_test = train_test_split(indep_X, dep_Y, test_size=0.30, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, Y_train, Y_test

def r2_prediction(regressor, X_test, Y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    return r2_score(Y_test, y_pred)

def Linear(X_train, Y_train, X_test):
    from sklearn.linear_model import Ridge
    param_dist = {'alpha': uniform(0, 10)}
    search = RandomizedSearchCV(Ridge(), param_distributions=param_dist,
                                n_iter=10, scoring='r2', cv=3, random_state=42)
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def svm_linear(X_train, Y_train, X_test):
    from sklearn.svm import SVR
    param_dist = {'C': np.logspace(-2, 2, 5), 'epsilon': [0.01, 0.1, 0.2], 'kernel': ['linear']}
    search = RandomizedSearchCV(SVR(), param_distributions=param_dist, n_iter=10,
                                cv=3, scoring='r2', random_state=42)
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def svm_NL(X_train, Y_train, X_test):
    from sklearn.svm import SVR
    param_dist = {'C': np.logspace(-2, 2, 5), 'gamma': ['scale', 0.01, 0.1, 1],
                  'epsilon': [0.01, 0.1, 0.2], 'kernel': ['rbf']}
    search = RandomizedSearchCV(SVR(), param_distributions=param_dist, n_iter=10,
                                cv=3, scoring='r2', random_state=42)
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def Decision(X_train, Y_train, X_test):
    from sklearn.tree import DecisionTreeRegressor
    param_dist = {'max_depth': [None, 5, 10, 20],
                  'min_samples_split': [2, 5, 10],
                  'min_samples_leaf': [1, 2, 4]}
    search = RandomizedSearchCV(DecisionTreeRegressor(random_state=42),
                                param_distributions=param_dist, n_iter=10,
                                cv=3, scoring='r2', random_state=42)
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def random(X_train, Y_train, X_test):
    from sklearn.ensemble import RandomForestRegressor
    param_dist = {'n_estimators': [100, 200],
                  'max_depth': [None, 10, 20],
                  'min_samples_split': [2, 5],
                  'min_samples_leaf': [1, 2]}
    search = RandomizedSearchCV(RandomForestRegressor(random_state=42),
                                param_distributions=param_dist, n_iter=10,
                                cv=3, scoring='r2', random_state=42)
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def XGBoost_Reg(X_train, Y_train, X_test):
    from xgboost import XGBRegressor
    params = {'n_estimators':[100,200], 'learning_rate':[0.01,0.05,0.1], 'max_depth':[3,5,7]}
    search = GridSearchCV(XGBRegressor(eval_metric='rmse'), params, cv=3, scoring='r2')
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def AdaBoost_Reg(X_train, Y_train, X_test):
    from sklearn.ensemble import AdaBoostRegressor
    params = {'n_estimators':[50,100,200], 'learning_rate':[0.01,0.1,1]}
    search = GridSearchCV(AdaBoostRegressor(), params, cv=3, scoring='r2')
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def LightGBM_Reg(X_train, Y_train, X_test):
    from lightgbm import LGBMRegressor
    params = {'num_leaves':[31,50], 'learning_rate':[0.01,0.05,0.1], 'n_estimators':[100,200]}
    model = LGBMRegressor(verbose=-1)
    search = GridSearchCV(LGBMRegressor(), params, cv=3, scoring='r2')
    search.fit(X_train, Y_train)
    return r2_prediction(search.best_estimator_, X_test, Y_test)

def selectk_regression(*scores):
    models = ['Linear', 'SVMl', 'SVMnl', 'Decision', 'Random', 'XGB', 'ADA', 'LGB']
    df = pd.DataFrame([scores], index=['Score'], columns=models)
    return df

In [10]:
kbest = selectkbest(indep_X, dep_Y, 6)
X_train, X_test, Y_train, Y_test = split_scalar(kbest, dep_Y)

scores = [
    Linear(X_train, Y_train, X_test),
    svm_linear(X_train, Y_train, X_test),
    svm_NL(X_train, Y_train, X_test),
    Decision(X_train, Y_train, X_test),
    random(X_train, Y_train, X_test),
    XGBoost_Reg(X_train, Y_train, X_test),
    AdaBoost_Reg(X_train, Y_train, X_test),
    LightGBM_Reg(X_train, Y_train, X_test),
]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000025 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 125
[LightGBM] [Info] Number of data points in the train set: 93, number of used features: 6
[LightGBM] [Info] Start training from score 55385.446237
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [6]:
result1 = selectk_regression(*scores)
print(result1)
#5

        Linear      SVMl     SVMnl  Decision    Random       XGB       ADA  \
Score  0.24524  0.114094 -0.004692  0.008471  0.137404  0.065568  0.100638   

           LGB  
Score  0.13207  


In [9]:
result2 = selectk_regression(*scores)
print(result2)
#4

         Linear      SVMl     SVMnl  Decision    Random       XGB       ADA  \
Score  0.212954  0.097613 -0.005042 -0.019413  0.131442  0.050857  0.124613   

            LGB  
Score  0.126034  


In [11]:
result3 = selectk_regression(*scores)
print(result3)
#6

         Linear      SVMl    SVMnl  Decision    Random      XGB       ADA  \
Score  0.231279  0.112549 -0.00739  0.008471  0.081675  0.06246  0.111721   

            LGB  
Score  0.084467  


In [12]:
results = [result1, result2, result3]   # add as many as we have

summary = []

for i, df in enumerate(results, start=5):   # since numbering starts at #5
    best_model = df.max().idxmax()          # best column name (model)
    best_score = df.max().max()             # best R² or accuracy
    summary.append({
        "Result_No": f"# {i}",
        "Best_Model": best_model,
        "Best_Score": best_score
    })

summary_df = pd.DataFrame(summary)
print(summary_df)

  Result_No Best_Model  Best_Score
0       # 5     Linear    0.245240
1       # 6     Linear    0.212954
2       # 7     Linear    0.231279
